Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [11]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
hidden_layer_size = 1024
beta = 0.007

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases1 = tf.Variable(tf.zeros([hidden_layer_size]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1 
  hidden_layer_output = tf.nn.relu(logits1)
  a2 = tf.matmul(hidden_layer_output, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(a2, tf_train_labels))
  loss += beta * tf.nn.l2_loss(weights1) + beta * tf.nn.l2_loss(weights2)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(a2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)
  init = tf.initialize_all_variables()

In [12]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])


with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  session.run(init)
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Loss at step 0: 2515.682861
Training accuracy: 11.3%
Validation accuracy: 33.0%
Loss at step 100: 1085.302734
Training accuracy: 92.4%
Validation accuracy: 79.3%
Loss at step 200: 537.278625
Training accuracy: 99.5%
Validation accuracy: 79.0%
Loss at step 300: 266.498077
Training accuracy: 99.8%
Validation accuracy: 79.4%
Loss at step 400: 132.252533
Training accuracy: 99.8%
Validation accuracy: 80.2%
Loss at step 500: 65.740807
Training accuracy: 99.7%
Validation accuracy: 81.8%
Loss at step 600: 32.833366
Training accuracy: 99.1%
Validation accuracy: 83.2%
Loss at step 700: 16.565762
Training accuracy: 97.7%
Validation accuracy: 84.3%
Loss at step 800: 8.521651
Training accuracy: 96.2%
Validation accuracy: 84.6%
Test accuracy: 91.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [6]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 100
hidden_layer_size = 1024
beta = 0.007
num_steps = 801

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases1 = tf.Variable(tf.zeros([hidden_layer_size]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1 
  hidden_layer_output = tf.nn.relu(logits1)
  a2 = tf.matmul(hidden_layer_output, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(a2, tf_train_labels))
  loss += beta * tf.nn.l2_loss(weights1) + beta * tf.nn.l2_loss(weights2)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(a2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)
  init = tf.initialize_all_variables()

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])


with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  session.run(init)
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Loss at step 0: 2587.625732
Training accuracy: 10.0%
Validation accuracy: 29.2%
Loss at step 100: 1093.499023
Training accuracy: 100.0%
Validation accuracy: 56.2%
Loss at step 200: 542.349548
Training accuracy: 100.0%
Validation accuracy: 56.3%
Loss at step 300: 268.992401
Training accuracy: 100.0%
Validation accuracy: 56.3%
Loss at step 400: 133.413788
Training accuracy: 100.0%
Validation accuracy: 56.5%
Loss at step 500: 66.170631
Training accuracy: 100.0%
Validation accuracy: 57.2%
Loss at step 600: 32.822388
Training accuracy: 100.0%
Validation accuracy: 58.5%
Loss at step 700: 16.296944
Training accuracy: 100.0%
Validation accuracy: 61.2%
Loss at step 800: 8.129767
Training accuracy: 100.0%
Validation accuracy: 63.3%
Test accuracy: 69.7%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [10]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 100
hidden_layer_size = 1024
beta = 0.007
num_steps = 801
keep_prob = 0.8

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases1 = tf.Variable(tf.zeros([hidden_layer_size]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1 
  hidden_layer_output = tf.nn.relu(logits1)
  tf.nn.dropout(hidden_layer_output, keep_prob)  
    
  a2 = tf.matmul(hidden_layer_output, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(a2, tf_train_labels))
  loss += beta * tf.nn.l2_loss(weights1) + beta * tf.nn.l2_loss(weights2)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(a2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)
  init = tf.initialize_all_variables()

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])


with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  session.run(init)
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 2531.290039
Training accuracy: 13.0%
Validation accuracy: 30.2%
Loss at step 100: 1095.070312
Training accuracy: 100.0%
Validation accuracy: 61.5%
Loss at step 200: 543.128235
Training accuracy: 100.0%
Validation accuracy: 61.6%
Loss at step 300: 269.378571
Training accuracy: 100.0%
Validation accuracy: 61.7%
Loss at step 400: 133.605347
Training accuracy: 100.0%
Validation accuracy: 61.9%
Loss at step 500: 66.265610
Training accuracy: 100.0%
Validation accuracy: 62.2%
Loss at step 600: 32.869442
Training accuracy: 100.0%
Validation accuracy: 62.5%
Loss at step 700: 16.321352
Training accuracy: 100.0%
Validation accuracy: 64.2%
Loss at step 800: 8.147290
Training accuracy: 100.0%
Validation accuracy: 64.8%
Test accuracy: 71.6%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [6]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
hidden_layer_size = 1024
beta = 0.007
num_steps = 801
keep_prob = 1

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases1 = tf.Variable(tf.zeros([hidden_layer_size]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1 
  hidden_layer_output = tf.nn.relu(logits1)
  tf.nn.dropout(hidden_layer_output, keep_prob)  
    
  a2 = tf.matmul(hidden_layer_output, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(a2, tf_train_labels))
  loss += beta * tf.nn.l2_loss(weights1) + beta * tf.nn.l2_loss(weights2)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(a2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)
  init = tf.initialize_all_variables()

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])


with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  session.run(init)
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 2506.126465
Training accuracy: 13.2%
Validation accuracy: 42.1%
Loss at step 100: 1085.871338
Training accuracy: 92.5%
Validation accuracy: 78.5%
Loss at step 200: 537.884644
Training accuracy: 99.4%
Validation accuracy: 78.2%
Loss at step 300: 266.781128
Training accuracy: 99.8%
Validation accuracy: 78.9%
Loss at step 400: 132.385513
Training accuracy: 99.8%
Validation accuracy: 79.7%
Loss at step 500: 65.801498
Training accuracy: 99.7%
Validation accuracy: 81.3%
Loss at step 600: 32.860462
Training accuracy: 99.2%
Validation accuracy: 82.6%
Loss at step 700: 16.577105
Training accuracy: 97.9%
Validation accuracy: 84.0%
Loss at step 800: 8.524786
Training accuracy: 96.3%
Validation accuracy: 84.5%
Test accuracy: 91.3%


# 2 hidden layers

In [ ]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
hidden_layer_1_size = 1024
hidden_layer_2_size = 200
beta = 0.007
num_steps = 801
keep_prob = 1

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_1_size]))
  biases1 = tf.Variable(tf.zeros([hidden_layer_1_size]))
  weights2 = tf.Variable(tf.truncated_normal([hidden_layer_1_size, hidden_layer_2_size]))
  biases2 = tf.Variable(tf.zeros([hidden_layer_2_size])) 
  weights3 = tf.Variable(tf.truncated_normal([hidden_layer_2_size, num_labels]))
  biases3 = tf.Variable(tf.zeros([num_labels]))

  def model(data):
    hidden_layer_1_output = tf.matmul(data, weights1) + biases1 
    hidden_layer_1_output = tf.nn.relu(hidden_layer_1_output)
    #tf.nn.dropout(hidden_layer_1_output, keep_prob)  
    
    hidden_layer_2_output = tf.matmul(hidden_layer_1_output, weights2) + biases2
    hidden_layer_2_output = tf.nn.relu(hidden_layer_2_output)

    output = tf.matmul(hidden_layer_2_output, weights3) + biases3
    return output

  output = model(tf_train_dataset)
  cross_entropy = (-tf.reduce_sum(tf_train_labels * tf.log(tf.clip_by_value(tf.nn.softmax(output),1e-10,1.0))))/train_subset
  loss = tf.reduce_mean(cross_entropy)
  loss += beta * tf.nn.l2_loss(weights1) + beta * tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(output)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))
  init = tf.initialize_all_variables()

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])


with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  session.run(init)
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 3548.113770
Training accuracy: 10.3%
Validation accuracy: 14.3%
Loss at step 100: 1375.156128
Training accuracy: 10.5%
Validation accuracy: 10.0%
Loss at step 200: 691.086060
Training accuracy: 15.2%
Validation accuracy: 10.0%
Loss at step 300: 333.715302
Training accuracy: 74.8%
Validation accuracy: 73.4%
Loss at step 400: 166.159393
Training accuracy: 76.1%
Validation accuracy: 73.4%
Loss at step 500: 83.016319
Training accuracy: 81.4%
Validation accuracy: 79.5%
